In [101]:
import numpy as np
from scipy.linalg import lstsq
import matplotlib.pyplot as plt
import plotly.graph_objs as go
import open3d as o3d


file = "./results/ex5rot_predicted.ply"

pcd = o3d.io.read_point_cloud(file)
xyz = np.array(pcd.points)

# left_arm = [627, 628, 789, 1232, 1233, 1311, 1315, 1378, 1379, 1381, 1382, 1385, 1388, 1389, 1393, 1394, 1396, 1397]
left_calf = [1087, 1088, 1091, 1092, 1096, 1097, 1099, 1100, 1103, 1155, 1371, 1464, 1467, 1469, 1528]
xyz = xyz[left_calf]

In [102]:
fig = go.Figure(data=[go.Scatter3d(x=xyz[:, 0], y=xyz[:, 1], z=xyz[:, 2], mode="markers", marker=dict(size=5))])
fig.update_layout(scene=dict(xaxis_title="X", yaxis_title="Y", zaxis_title="Z"), title="3D Scatter Plot")
fig.show()

In [103]:
from skspatial.objects import Plane, Points

points = Points(xyz)
plane = Plane.best_fit(points)
normal = np.array(plane.normal)

In [104]:
y = np.array([0, 1, 0])
phi = np.arccos(np.dot(normal, y) / (np.linalg.norm(normal) * np.linalg.norm(y)))
theta = -np.arctan2(normal[0], normal[2])

rotZ = np.array([[np.cos(phi), -np.sin(phi), 0], [np.sin(phi), np.cos(phi), 0], [0, 0, 1]])
rotY = np.array([[np.cos(theta), 0, np.sin(theta)], [0, 1, 0], [-np.sin(theta), 0, np.cos(theta)]])

In [105]:
for i in range(xyz.shape[0]):
    xyz[i] = np.array(plane.project_point(xyz[i]))

for i in range(xyz.shape[0]):
    xyz[i] = np.dot(rotY, xyz[i])

In [106]:
fig = go.Figure(data=[go.Scatter3d(x=xyz[:, 0], y=xyz[:, 1], z=xyz[:, 2], mode="markers", marker=dict(size=5))])
fig.update_layout(scene=dict(xaxis_title="X", yaxis_title="Y", zaxis_title="Z"), title="3D Scatter Plot")
fig.show()

In [63]:
plt.plot(xz[:, 0], xz[:, 1], ".")
plt.show()

NameError: name 'xz' is not defined

In [ ]:
# Generate sample data (replace this with your own 3D points)
points = xyz

# Define the design matrix A
A = np.column_stack((points[:, 0], points[:, 1], np.ones(points.shape[0])))

# Solve for the parameters using lstsq
params, _, _, _ = lstsq(A, points[:, 2])

In [ ]:
params

In [ ]:
from scipy.spatial import ConvexHull


def compute_convex_hull(points):
    """
    Compute the convex hull from a 2D numpy array of points.

    Parameters:
        points (numpy.ndarray): Array of shape (n, 2) representing 2D points.

    Returns:
        numpy.ndarray: Array of shape (m, 2) representing the vertices of the convex hull.
    """
    # Ensure the input is a numpy array
    points = np.array(points)

    # Compute the convex hull
    hull = ConvexHull(points)

    # Get the vertices of the convex hull
    hull_vertices = points[hull.vertices]

    return hull_vertices

In [ ]:
xz = xyz[:, [0, 2]]

In [ ]:
convex_hull = compute_convex_hull(xz)

In [ ]:
plt.plot(xz[:, 0], xz[:, 1], ".")
plt.plot(convex_hull[:, 0], convex_hull[:, 1], ".")
plt.show()

In [ ]:
length = 0

for i in range(1, convex_hull.shape[0]):
    length += np.linalg.norm(convex_hull[i] - convex_hull[i - 1])

length += np.linalg.norm(convex_hull[-1] - convex_hull[0])

In [ ]:
convex_hull.shape[0]

In [ ]:
length * 1.73 / (0.84 + 0.95)

In [ ]:
(0.84 + 0.95) / 1.73

In [ ]:
import numpy as np
from scipy.linalg import lstsq

# Generate sample data (replace this with your own 3D points)
points = np.random.rand(10, 3)

# Define the design matrix A
A = np.column_stack((points[:, 0], points[:, 1], np.ones(points.shape[0])))

# Solve for the parameters using lstsq
params, _, _, _ = lstsq(A, points[:, 2])

In [ ]:
params

In [ ]:
import numpy as np
from scipy.linalg import lstsq

# Generate sample data (replace this with your own 3D points)
points = xyz

# Define the design matrix A
A = np.column_stack((points[:, 0], points[:, 1], points[:, 2], np.ones(points.shape[0])))

# Solve for the parameters using lstsq
params, _, _, _ = lstsq(A, -np.ones(points.shape[0]))

# Extract parameters
a, b, c, d = params

# Print the parameters
print("Parameters of the plane fitting the points:")
print(f"a: {a}, b: {b}, c: {c}, d: {d}")